## 拟合一个线性模型

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'1.12.0'

In [3]:
# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300
print(x_data)

[[0.57971233 0.6922096  0.1862811  0.08015992 0.35493222 0.4533392
  0.30138177 0.86929256 0.24062529 0.73986554 0.12306147 0.4170585
  0.458017   0.18537052 0.43813488 0.32337582 0.13075176 0.99533564
  0.2485816  0.21569179 0.8824464  0.7476094  0.19890445 0.58330977
  0.2845167  0.33731416 0.805028   0.2911561  0.772349   0.21166809
  0.6303933  0.43204618 0.2580785  0.10473256 0.11568049 0.51444876
  0.04711734 0.2218198  0.47130364 0.20331097 0.8370063  0.87278444
  0.5159315  0.6894737  0.65955335 0.7849121  0.71792483 0.88950175
  0.79269344 0.05527392 0.17011935 0.09333815 0.80044436 0.4726555
  0.17023045 0.496394   0.88364995 0.494009   0.95980906 0.03053826
  0.8335576  0.94249684 0.2912182  0.17577913 0.27101573 0.12830995
  0.36021486 0.5309375  0.7657706  0.8277268  0.1120047  0.73388976
  0.7685977  0.05426965 0.06542509 0.6277729  0.4110911  0.07423472
  0.6961466  0.6614386  0.35282713 0.21275187 0.81180084 0.87481844
  0.66012114 0.1100129  0.15025774 0.49090236 0.945

In [4]:
# 构造一个线性模型 
with tf.device("/gpu:0"):
    b = tf.Variable(tf.zeros([1]))
    W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
    y = tf.matmul(W, x_data) + b

    # 最小化方差
    loss = tf.reduce_mean(tf.square(y - y_data))
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = optimizer.minimize(loss)

    # 初始化变量
    # init = tf.initialize_all_variables()
    init = tf.global_variables_initializer()

## 运行会话的第一种方式，用上下文管理（推荐）

In [5]:
# allow_soft_placement=True 没有GPU的话在CPU上运行
# tf 默认会一次性占用所有内存
# 动态分配内存的方法
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
# config.gpu_options.allow_growth = True #  按需分配显存
## 按照固定的比例分配。
## 以下代码会占用所有可使用的GPU的40%显存
config.gpu_options.per_process_gpu_memory_fraction = 0.4

with tf.Session(config=config) as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for step in range(0, 101):
        sess.run(train)
        if step % 20 == 0:
            print("Step:{} | W:{} b:{}| Loss:{}".format(step, sess.run(W), sess.run(b), sess.run(loss)))      

Step:0 | W:[[0.2649486  0.05522385]] b:[0.60009617]| Loss:0.09838163107633591
Step:20 | W:[[0.08773954 0.14677931]] b:[0.33402646]| Loss:0.00024953047977760434
Step:40 | W:[[0.09083804 0.18582936]] b:[0.31199697]| Loss:2.220932401542086e-05
Step:60 | W:[[0.09607617 0.19558087]] b:[0.3042725]| Loss:2.6739771783468314e-06
Step:80 | W:[[0.09850787 0.19850503]] b:[0.30152738]| Loss:3.401148660486797e-07
Step:100 | W:[[0.09945375 0.19947621]] b:[0.30054682]| Loss:4.361079319892269e-08


## 我的设备上的GPU占用果然按需分配
```
gz_6237_gpu             Sat Feb 15 22:56:50 2020  418.87.00
[0] GeForce RTX 2080 Ti | 46'C,   0 % |   295 / 10989 MB | dc:python/944(285M)
```

## 0.4 验证
```
gz_6237_gpu             Sat Feb 15 23:01:56 2020  418.87.00
[0] GeForce RTX 2080 Ti | 43'C,   0 % |  4691 / 10989 MB | dc:python/1641(4681M)
```

In [6]:
## 按照固定的比例分配。
## 以下代码会占用所有可使用的GPU的40%显存
#config.gpu_optionsu.pre_process_gpu_memery_fraction = 0.4

## 运行会话的第二种方式（不推荐）

In [7]:
# 启动图 (graph)
sess = tf.Session(config = tf.ConfigProto(log_device_placement=True))
sess.run(init)

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print("Step:{} | W:{} b:{}| Loss:{}".format(step, sess.run(W), sess.run(b), sess.run(loss)))
sess.close() # 必须关闭

Step:0 | W:[[ 0.865156   -0.54459083]] b:[0.6344692]| Loss:0.215562641620636
Step:20 | W:[[0.16806549 0.0657469 ]] b:[0.33653265]| Loss:0.0021513639949262142
Step:40 | W:[[0.10180449 0.17509796]] b:[0.3121652]| Loss:5.5768032325431705e-05
Step:60 | W:[[0.09761121 0.19419566]] b:[0.3042347]| Loss:3.178116458002478e-06
Step:80 | W:[[0.09873608 0.19833933]] b:[0.30150068]| Loss:3.361149651937012e-07
Step:100 | W:[[0.09949227 0.19946127]] b:[0.30053547]| Loss:4.185685753554935e-08
Step:120 | W:[[0.0998107 0.1998145]] b:[0.3001916]| Loss:5.352108534140143e-09
Step:140 | W:[[0.09993116 0.19993453]] b:[0.30006862]| Loss:6.871650470863244e-10
Step:160 | W:[[0.09997521 0.19997668]] b:[0.30002457]| Loss:8.807348600425868e-11
Step:180 | W:[[0.09999108 0.19999166]] b:[0.30000883]| Loss:1.1372743025095389e-11
Step:200 | W:[[0.09999678 0.19999701]] b:[0.30000317]| Loss:1.4702194539931157e-12


In [11]:
import os

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # 指定只是用第三块GPU